In [37]:
from music21 import converter, instrument, note, chord
import glob
import numpy
import pickle

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# Training

In [45]:
def train_network():
    notes = get_notes()
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    model = create_network(network_input, n_vocab)
    
    train(model,network_input, network_output)


In [46]:
def get_notes():

    notes = []
    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)
        notes_to_parse = None
        
        try: #the notes are instrument whole notes to parse

            s2 = instrument.partitionByInstrument(midi)

            notes_to_parse = s2.parts[0].recurse()
        except: # if the file has notes in a flat structure
            notes_to_parse.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                
    with open('notes','wb') as filepath:
        pickle.dump(notes,filepath)
    
    return notes



In [48]:
#mapping the string based categorical data into an integer-based numerical data

# Create input sequences for hte network and their respective outputs

def prepare_sequences():

    sequence_length = 100

    # get all pitch names 
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []


    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_output)

    #reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

    #normalize iput

    network_input = network_input/float(n_vocab)

    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)



# Model

In [49]:
# LSTM layers is a RNN that takes a sequence as input and can return
# either seqeunces (return_sequences = True) or a matrix

In [50]:
# Dropout layers are a regularization otechnique that consists
# of setting a fraction of input units to 0 at each update
# during the training to prevent overfitting. The fraction is 
# determined by the parameter used with the layer.



In [51]:
# Dense layers or fully connected layers is a fully connected 
# neural network layer where each input node is connected to
# each output node.


In [52]:
# The Activation layer determines what activation function our
# neural network will use to calculate the output of a node.

In [53]:
def create_network(network_input, n_vocab):

    model = Sequential()
    model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]),
                  return_sequences = True
                  ))

    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences = True))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer = 'rmsprop')
    
    return model

In [56]:
def train(model, network_input, network_output):
    filepath = 'weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5'
    checkpoint = ModelCheckpoint(
        filepath, monitor='loss', 
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
if __name__ == '__main__':
    train_network()

TypeError: prepare_sequences() takes 0 positional arguments but 2 were given

In [ ]:
#Reuse training code to load weights

model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights.hdf5')

In [ ]:
# Note Generation using Trained Notes

start = numpy.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

#generate 500 notes

for note_index in range(500):
    prediction_input = numpy.reshape(pattern(1, len(pattern), 1))
    prediction_input = prediction_input/float(n_vocab)
    
    prediction = model.predict(prediction_input, verbose=0)
    
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [29]:
# The first sequence we submit is the sequence of notes 
# at the starting index. For every subsequent sequence that 
# we use as input, we will remove the first note of the sequence 
# and insert the output of the previous iteration at the end of 
# the sequence.

In [32]:
# To determine the most likely prediction from the output 
# from the network, we extract the index of the highest value. 
# The value at index X in the output array correspond to the 
# probability that X is the next note.

In [33]:
# First we have to determine whether the output we are decoding is 
# a Note or a Chord.
# If the pattern is a Chord, we have to split the string up 
# into an array of notes. Then we loop through the string 
# representation of each note and create a Note object for 
# each of them. Then we can create a Chord object containing 
# each of these notes.


# If the pattern is a Note, we create a Note object using the 
# string representation of the pitch contained in the pattern.

# At the end of each iteration we increase the offset by 0.5 
# (as we decided in a previous section) and append the Note/Chord 
# object created to a list.

In [ ]:
offset = 0
output_notes = []

#create note and chord objects - based on values from the model

for pattern in prediction_output:
    #pattern is a chord
    if('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    #pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    #increase offset each iteration so that the notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')